### Chains using LangGraph

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os

groq_api_key = os.getenv("GROQ_API_KEY")

In [5]:
from langchain_core.messages import AIMessage, HumanMessage
from pprint import pprint

messages= [AIMessage(content="how can I help you", name="Model")]
messages.append(HumanMessage(content="Hi, I am Mounica", name="User"))
messages.append(AIMessage(content="Hi Mounica, how may I help", name="Model"))
messages.append(HumanMessage(content="Im planning to learn langgraph", name="User"))

for message in messages:
    message.pretty_print()


================================== Ai Message ==================================
Name: Model

how can I help you
================================ Human Message =================================
Name: User

Hi, I am Mounica
================================== Ai Message ==================================
Name: Model

Hi Mounica, how may I help
================================ Human Message =================================
Name: User

Im planning to learn langgraph


#### Chat Model

In [7]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.1-8b-instant")
response=model.invoke(messages)
response.pretty_print()

================================== Ai Message ==================================

You're interested in learning Langgraph. Can you please provide more context or information about Langgraph? I'd be happy to help if you're looking for resources, tutorials, or general information about it.


In [8]:
response.response_metadata

{'token_usage': {'completion_tokens': 42,
  'prompt_tokens': 77,
  'total_tokens': 119,
  'completion_time': 0.081637631,
  'completion_tokens_details': None,
  'prompt_time': 0.00557929,
  'prompt_tokens_details': None,
  'queue_time': 0.05009183,
  'total_time': 0.087216921},
 'model_name': 'llama-3.1-8b-instant',
 'system_fingerprint': 'fp_4387d3edbb',
 'service_tier': 'on_demand',
 'finish_reason': 'stop',
 'logprobs': None,
 'model_provider': 'groq'}

#### Tools

* Tools can be integrated with LLM Models to interact with external systems. External systems can be API's, third party tools.